In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
slow_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
save_path = '/content/drive/MyDrive/2040 final/2040 final proj/bert_base_uncased'
slow_tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/2040 final/2040 final proj/bert_base_uncased/tokenizer_config.json',
 '/content/drive/MyDrive/2040 final/2040 final proj/bert_base_uncased/special_tokens_map.json',
 '/content/drive/MyDrive/2040 final/2040 final proj/bert_base_uncased/vocab.txt',
 '/content/drive/MyDrive/2040 final/2040 final proj/bert_base_uncased/added_tokens.json')

In [ ]:
tokenizer = BertWordPieceTokenizer('/content/drive/MyDrive/2040 final/2040 final proj/bert_base_uncased/vocab.txt', lowercase=True)

In [ ]:
import numpy as np
import json
import pandas as pd
import os
import random
import datetime
import re
import math
import pickle
import transformers
import datasets
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from functools import reduce
import gensim

data_files = ['azn', 'biontech', 'jnj', 'moderna', 'novavax', 'pfizer']
data_files = ['jnj']

def save_obj(obj, name):
    """
    :param obj:
    :param name:
    :return:
    """
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name):
    """
    :param name:
    :return:
    """
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


def list_major_and_related(dir):
    """
    :param dir:
    :return:
    """
    files = os.listdir(dir)
    major_tweets_f = []
    related_tweets_f = []
    for fn in files:
        if 'related' in fn:
            related_tweets_f.append(fn)
        else:
            major_tweets_f.append(fn)
    return major_tweets_f, related_tweets_f


def read_file(n):
    """
    :param n:
    :return:
    """
    with open(n, 'rb') as f:
        fr = f.readlines()

    def decoder(s):
        try:
            ds = str(s, 'utf_8_sig')
        except UnicodeDecodeError:
            try:
                ds = str(s, 'utf-8')
            except UnicodeDecodeError:
                try:
                    ds = str(s, 'gbk')
                except UnicodeDecodeError:
                    try:
                        ds = str(s, 'GB18030')
                    except UnicodeDecodeError:
                        ds = str(s, 'gb2312', 'ignore')
        return ds

    fr = list(map(lambda x: json.loads(decoder(x)), fr))
    return fr


def process_file(file_dir: str,
                 company_name: str,
                 abandon: list = None) -> pd.DataFrame:
    """
    Processing the related twitters regarding company_name
    :param file_dir: file directory
    :param company_name: target company name
    :param abandon: the columns to abandon
    :return: a pandas dataframe of cleaned data
    """
    if abandon is None:
        abandon = ['conversation_id', 'created_at', 'time', 'timezone', 'user_id',
                   'username', 'name', 'place', 'mentions', 'link', 'quote_url',
                   'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
                   'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']
    op = read_file(file_dir)
    d = pd.DataFrame(op)
    d = d.drop_duplicates(subset=['id'])
    d = d.drop(columns=abandon)
    d = d[d['language'] == 'en'].reset_index(drop=True)
    d = d.drop(columns=['language'])
    # process date
    d['date'] = d['date'].apply(lambda x: x.replace('-', ''))
    # process urls:
    d['linked_url'] = d['urls'].apply(lambda x: 1 if x else 0)
    d = d.drop(columns=['urls'])
    # process photos:
    d['with_photo'] = d['photos'].apply(lambda x: 1 if x else 0)
    d = d.drop(columns=['photos'])
    # process retweet
    d['retweet'] = d['retweet'].apply(lambda x: 1 if x else 0)
    # process video
    d['video'] = d['video'].apply(lambda x: 1 if x else 0)
    # process date:
    d['date'] = d['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d'))
    d['date'] = d['date'].apply(
        lambda x: x + datetime.timedelta(days=-1) if x.weekday() == 5 else x + datetime.timedelta(
            days=-2) if x.weekday() == 6 else x)
    d['date'] = d['date'].apply(lambda x: datetime.datetime.strftime(x, '%Y%m%d'))
    # process format:
    d['replies_count'] = d['replies_count'].astype('int64')
    d['retweets_count'] = d['retweets_count'].astype('int64')
    d['likes_count'] = d['likes_count'].astype('int64')
    d['retweet'] = d['retweet'].astype('int64')
    d['video'] = d['video'].astype('int64')
    d['linked_url'] = d['linked_url'].astype('int64')
    d['with_photo'] = d['with_photo'].astype('int64')
    # group by date
    new_df = [dict(zip(sub_df.columns, [sum(sub_df[x])
                                        if sub_df[x].dtype == 'int64'
                                        else sub_df[x].tolist()
                                        for x in sub_df.columns]))
              for _, sub_df in d.groupby('date')]
    new_df = pd.DataFrame(new_df)
    new_df['company_name'] = company_name
    new_df['date'] = new_df['date'].apply(lambda x: x[0])
    return new_df


# process json file for each company
def gather_all(dfs: list = None) -> pd.DataFrame:
    """
    :param dfs:
    :return:
    """
    if dfs is None:
        dfs = data_files
    corp_gathered = []
    for f in dfs:
        mtf, rtf = list_major_and_related('/content/drive/MyDrive/2040 final/2040 final proj/{}'.format(f))
        print('...Processing file {}...{}'.format(f, rtf))
        corp_file = []
        for j in rtf + mtf:
            j_d = '/content/drive/MyDrive/2040 final/2040 final proj/{}/'.format(f) + j
            corp_file.append(process_file(file_dir=j_d,
                                          company_name=f))
        corp_file = reduce(lambda x, y: pd.concat([x, y], ignore_index=True), corp_file)
        corp_gathered.append(corp_file)
    corp_gathered = reduce(lambda x, y: pd.concat([x, y], ignore_index=True), corp_gathered)
    corp_gathered = corp_gathered.drop_duplicates(subset=['date', 'company_name'], keep='first')
    return corp_gathered


def cum_sum(target):
    """
    :param target:
    :return:
    """
    s = sum(target)
    r = list(range(s))
    res = []
    for i in target:
        tmp = r[0:i]
        r = r[i:]
        res.append(tmp)
    return res


class ProcessR(object):
    def __init__(self,
                 text,
                 filters='',
                 max_len=384):
        """
        :param text:
        :param filters:
        :param padding:
        """
        self.text = text
        self.max_len = max_len
        self.filters = filters
        print('...{} Documents Loaded...'.format(len(self.text)))
        self.docs = self._preprocess_docs()
        print('...Language Preprocess Finished...')
        self.docs_f = self._tokenize()
        print('...Language Tokenizing Finished...')
        self.bert = self._initialize_bert_model()
        print('...BERT Model Initialized...')
        print('...Inferring via BERT Model...')
        self.bert_vec = self._bert_process()
        print('...Finished...')

    def _preprocess_docs(self):
        def _process_single(text):
            text = re.sub(r'http\S+', '', text)
            text = re.sub(r'@\S+', '', text)
            text = re.sub(r':\S+', '', text)
            text = re.sub('[0-9]+', '', text)
            return text
        return [_process_single(x) for x in self.text]
    
    def _padding(self, x):
        padding_length = self.max_len - len(x)
        if padding_length > 0:
            x = x + ([0] * padding_length)
        else:
            x = x[:self.max_len]
        return x

    def _tokenize(self):
        docs_ids = [tokenizer.encode(x).ids for x in self.docs]
        return [self._padding(x) for x in docs_ids]

    def _initialize_bert_model(self):
        encoder = TFBertModel.from_pretrained('bert-base-uncased')
        word_input = tf.keras.Input(shape=self.max_len, dtype='int64', name='word_input')
        word_embedding = encoder(word_input, token_type_ids=None, attention_mask=None)[0]
        vec_sum_1 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(word_embedding)
        word_level_model = tf.keras.Model(inputs=word_input, outputs=vec_sum_1)
        word_level_model.compile()
        return word_level_model

    def _bert_process(self):
        return [self.bert.predict(np.reshape(x, (1, self.max_len))).tolist() for x in self.docs_f]


def generate_tf_rec(f_m : pd.DataFrame,
                    sequence_length: int,
                    num_per_day: int,
                    n_repeat: int,
                    test_size: float):
    """
    :param test_size:
    :param n_repeat:
    :param num_per_day:
    :param processor:
    :param news_info:
    :param stock_info:
    :param map_relation:
    :param sequence_length:
    :return:
    """
    x, y = df_process(f_m)
    f_m['target_variation'] = y
    for company, sub_df in f_m.groupby('company_name'):
        print('Processing company {}'.format(company))
        train_writer = tf.io.TFRecordWriter(path='/content/drive/MyDrive/2040 final/2040 final proj/{}_train.tfrecords'.format(company))
        validation_writer = tf.io.TFRecordWriter(path='/content/drive/MyDrive/2040 final/2040 final proj/{}_validation.tfrecords'.format(company))
        tmp = sub_df.sort_values(by=['date'], ascending=True)
        sub_x = x[tmp.index, :]
        tmp = tmp.reset_index(drop=True)
        train_index = range(math.floor((tmp.shape[0] - sequence_length + 1)*(1-test_size)))
        date_text_related = dict(zip(tmp.date + '_' + tmp.company_name, cum_sum([len(eval(x)) for x in tmp.tweet.tolist()])))
        text_related = sum([eval(x) for x in tmp.tweet.tolist()], [])
        related_pr = ProcessR(text_related)
        for i in range(tmp.shape[0] - sequence_length):
            fluctuation_norm = tmp.iloc[i+sequence_length, :]['target_variation']
            sequence_date = tmp.iloc[i:i+sequence_length, :]['date']
            sequence_x = sub_x[i:i+sequence_length, :]
            for _ in range(random.randint(n_repeat - 10, n_repeat + 10)):
                sub_rec = []
                for d in sequence_date:
                    word_v = [related_pr.bert_vec[x] for x in date_text_related['{}_{}'.format(d, company)]]
                    word_v = [word_v[x] for x in [random.randint(0, len(word_v)-1) for _ in range(num_per_day)]]
                    sub_rec.append(word_v)
                sub_rec = np.array(sub_rec).astype('float32')
                sequence_x = sequence_x.astype('float32')
                example = tf.train.Example(features=tf.train.Features(
                    feature={
                        'x1': tf.train.Feature(bytes_list=tf.train.BytesList(value=[sub_rec.tobytes()])),
                        'x2': tf.train.Feature(bytes_list=tf.train.BytesList(value=[sequence_x.tobytes()])),
                        'y': tf.train.Feature(float_list=tf.train.FloatList(value=[fluctuation_norm]))
                    }
                ))
                if i in train_index:
                    train_writer.write(example.SerializeToString())
                else:
                    validation_writer.write(example.SerializeToString())
        train_writer.close()
        validation_writer.close()
    print('Finished')


def df_process(df,
               drop_columns=None,
               norm=True,
               norm_columns=None,
               norm_approach='min-max'):
    """
    :param df:
    :param drop_columns:
    :param norm:
    :param norm_columns:
    :param norm_approach:
    :param predict_time:
    :return:
    """
    if drop_columns is None:
        drop_columns = ['open', 'high', 'low', 'close',
                        'adjclose', 'volume', 'company_name', 'date',
                        'hashtags', 'cashtags', 'retweet']
    df = df.drop(columns=drop_columns)
    df = df.dropna()
    if norm:
        if norm_columns is None:
            norm_columns = ['replies_count', 'retweets_count',
                            'likes_count',
                            'video', 'linked_url', 'with_photo', 'target_variation']
        if norm_approach == 'min-max':
            s = MinMaxScaler()
        else:
            s = StandardScaler()
        s.fit(df[norm_columns])
        s_f = np.array(s.transform(df[norm_columns]))
    else:
        s_f = np.array(df)
    y = s_f[:, -1]
    x = s_f[:, :-1]
    return x, y

In [ ]:
gather_all().to_csv('/content/drive/MyDrive/2040 final/2040 final proj/data_op.csv', encoding='utf_8_sig', index=False)

...Processing file azn...['azn_related_202006202008.json', 'azn_related_201912202006.json', 'azn_related_20160401_20191201.json', 'azn_related_202008202101.json', 'azn_related_202101202104.json']
...Processing file biontech...['biontech_related_201912202006.json', 'bntx_related_20191010_20191201.json', 'biontech_related_202006202008.json', 'biontech_related_202008202010.json', 'biontech_related_202010202012.json', 'biontech_related_202012202102.json', 'biontech_related_202102202104.json']
...Processing file jnj...['jnj_related_201912202006.json', 'jnj_related_20160401_20191201.json', 'jnj_related_202006202101.json', 'jnj_related_202101202104.json']
...Processing file moderna...['moderna_related_201912202006.json', 'mrna_related_20181207_20191201.json', 'moderna_related_202006202010.json', 'moderna_related_202010202102.json', 'moderna_related_202102202104.json']
...Processing file novavax...['novavax_related_201912202006.json', 'nvax_related_20160401_20191201.json', 'novavax_related_202

In [ ]:
related_corp_tweets = pd.read_csv('/content/drive/MyDrive/2040 final/2040 final proj/data_op.csv', encoding='utf_8_sig')
stock_price = pd.read_csv('/content/drive/MyDrive/2040 final/2040 final proj/stockprice_201604_202006.csv', encoding='utf_8_sig', index_col=0)

related_corp_tweets['date'] = related_corp_tweets['date'].astype('str')
stock_price['date'] = stock_price['date'].astype('str')

In [ ]:
# mapping date to tweet
tweet_stock_merge = pd.merge(related_corp_tweets, stock_price, on=['company_name', 'date'], how='right')
tweet_stock_merge = tweet_stock_merge.dropna().reset_index(drop=True)
# date_text_related = dict(zip(tweet_stock_merge.date + '_' + tweet_stock_merge.company_name, cum_sum([len(eval(x)) for x in tweet_stock_merge.tweet.tolist()])))

In [ ]:
# format the texts
# text_related = sum([eval(x) for x in tweet_stock_merge.tweet.tolist()], [])

# related_pr = ProcessR(text_related)

In [ ]:
generate_tf_rec(f_m=tweet_stock_merge,
                sequence_length=10,
                num_per_day=50,
                n_repeat=20,
                test_size=0.2)

Processing company azn
...77704 Documents Loaded...
...Language Preprocess Finished...
...Language Tokenizing Finished...


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...BERT Model Initialized...
...Inferring via BERT Model...


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
related_corp_tweets = pd.read_csv('/content/drive/MyDrive/2040 final/2040 final proj/data_op.csv', encoding='utf_8_sig')
stock_price = pd.read_csv('/content/drive/MyDrive/2040 final/2040 final proj/stockprice_201604_202006.csv', encoding='utf_8_sig', index_col=0)

In [ ]:
related_corp_tweets['date'] = related_corp_tweets['date'].astype('str')
stock_price['date'] = stock_price['date'].astype('str')
# mapping date to tweet
tweet_stock_merge = pd.merge(related_corp_tweets, stock_price, on=['company_name', 'date'], how='right')
tweet_stock_merge = tweet_stock_merge.dropna().reset_index(drop=True)


In [ ]:
tweet_stock_merge.to_csv('/content/drive/MyDrive/2040 final/2040 final proj/data_20210408.csv', encoding='utf_8_sig', index=False)